In [ ]:
import numpy as np
import matplotlib.pyplot as plot
from scipy import interpolate

In [ ]:
chrisRawData = np.loadtxt("supernovadmplot.txt", skiprows=0, dtype="str")
chrisData = {}
for i in range(0,len(chrisRawData),2):
    chrisData[chrisRawData[i].lower()] = np.array(chrisRawData[i+1].split(",")).astype(float)

In [ ]:
myRawData = np.loadtxt("Oper_c1-0_IsotopicBreakdown.dat", skiprows=2, dtype="str")
isotopes = ["hydrogen", "helium", "carbon", "oxygen", "neon", "magnesium", "silicon", "sulfur", "iron"]
length = 1002
myData = {}
for i in range(len(isotopes)):
    isotopicData = []
    for j in range(1,length):
        isotopicData.append(myRawData[i*length+j][2])
    myData[isotopes[i].lower()] = np.array(isotopicData).astype(float)
vel = []
for i in range(1,length):
    vel.append(myRawData[i][1])
myData["velocity"] = np.array(vel).astype(float)

In [ ]:
plot.clf()
ax = plot.subplot(111)
colours = ["blue", "orange", "green", "red", "purple", "brown", "pink", "gray", "yellow"]
totalChris = np.zeros(chrisData["velocity"].shape)
totalSuper = np.zeros(myData["velocity"].shape)
for i in range(len(isotopes)):
    totalChris = totalChris + chrisData[isotopes[i]]
    plot.plot(chrisData["velocity"]*10**-5, chrisData[isotopes[i]], label="Chris "+isotopes[i], color=colours[i], linestyle="solid")
    totalSuper = totalSuper + myData[isotopes[i]]
    plot.plot(myData["velocity"], myData[isotopes[i]], label="Super "+isotopes[i], color=colours[i], linestyle="dashed")
plot.plot(chrisData["velocity"]*10**-5, totalChris, label="Chris Total", color="black", linestyle="solid")
plot.plot(myData["velocity"], totalSuper, label="Super Total", color="black", linestyle="dashed")

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plot.title(r"Isotopic velocity spectrum for $c_{1}^{0}$")
plot.xlabel(r"Dark Matter Velocity [km s$^{-1}$]")
plot.ylabel(r"Scattered Dark Matter[(cm s$^{-1}$)$^{-1}$ s$^{-1}$ cm$^{-2}$]")
# plot.xscale("log")
# plot.yscale("log")
# plot.ylim(top=0.0)
plot.xlim(4300, 4500)
plot.grid()
plot.savefig("isotopicBreakdown+total.pdf")

In [ ]:
plot.clf()
ax = plot.subplot(111)
colours = ["blue", "orange", "green", "red", "purple", "brown", "pink", "gray", "yellow"]
totalChris = np.zeros(chrisData["velocity"].shape)
totalSuper = np.zeros(myData["velocity"].shape)
for i in range(len(isotopes)):
    totalChris = totalChris + chrisData[isotopes[i]]
    # plot.plot(chrisData["velocity"]*10**-5, interpolate.interp1d(chrisData["velocity"]*10**-5, chrisData[isotopes[i]])(chrisData["velocity"]*10**-5), label="Chris "+isotopes[i], color=colours[i], linestyle="solid")
    totalSuper = totalSuper + myData[isotopes[i]]
    # plot.plot(myData["velocity"], interpolate.interp1d(myData["velocity"], myData[isotopes[i]])(myData["velocity"]), label="Super "+isotopes[i], color=colours[i], linestyle="dashed")
    plot.plot(myData["velocity"], interpolate.interp1d(myData["velocity"], myData[isotopes[i]])(myData["velocity"])/interpolate.interp1d(chrisData["velocity"]*10**-5, chrisData[isotopes[i]])(myData["velocity"]), label=isotopes[i], color=colours[i])
# plot.plot(chrisData["velocity"]*10**-5, interpolate.interp1d(chrisData["velocity"]*10**-5, totalChris)(chrisData["velocity"]*10**-5), label="Chris Total", color="black", linestyle="solid")
# plot.plot(myData["velocity"], interpolate.interp1d(myData["velocity"], totalSuper)(myData["velocity"]), label="Super Total", color="black", linestyle="dashed")
plot.plot(myData["velocity"], interpolate.interp1d(myData["velocity"], totalSuper)(myData["velocity"])/interpolate.interp1d(chrisData["velocity"]*10**-5, totalChris)(myData["velocity"]), label="Total", color="black")

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plot.title(r"Isotopic velocity spectrum for $c_{1}^{0}$ ratio Super/Chris")
plot.xlabel(r"Dark Matter Velocity [km s$^{-1}$]")
plot.ylabel(r"Scattered Dark Matter[(cm s$^{-1}$)$^{-1}$ s$^{-1}$ cm$^{-2}$]")
# plot.xscale("log")
# plot.yscale("log")
# plot.ylim(top=1.0)
plot.xlim(4300, 4500)
plot.grid()
plot.savefig("isotopicBreakdownRatio+total.pdf")